In [ ]:
import pandas as pd

# Load CVs
df_cv = pd.read_csv("data/CV_Extracted.csv")

# Load Job Descriptions
df_job = pd.read_csv("data/JobDescription.csv")

print("CVs Loaded:", len(df_cv))
print("Jobs Loaded:", len(df_job))

# Example: inspect first CV and Job
print(df_cv.head())
print(df_job.head())
